# Visualising networks using nglview

In [ ]:
import kugupu as kgp
import MDAnalysis as mda
import nglview as nv
import numpy as np

In [ ]:
res = kgp.load_results('datafiles/results.hdf5')

In [ ]:
def add_names(u):
    # Guesses atom names based upon masses
    def approx_equal(x, y):
        return abs(x - y) < 0.1
    
    # mapping of atom mass to element
    massdict = {}
    for m in set(u.atoms.masses):
        for elem, elem_mass in mda.guesser.tables.masses.items():
            if approx_equal(m, elem_mass):
                massdict[m] = elem
                break
        else:
            raise ValueError
            
    u.add_TopologyAttr('names')
    for m, e in massdict.items():
        u.atoms[u.atoms.masses == m].names = e

In [ ]:
u = mda.Universe('./datafiles/C6.data', './datafiles/C6.dcd')

In [ ]:
add_names(u)

## Visualising a network

Firstly an entire molecule network can be visualised using the `visualise.draw_networks` functions.

First we calculate the networks present in the first frame of results, here at a threshold of 1meV.

In [ ]:
nets = kgp.networks.find_networks(u.atoms.fragments, res.H_frag[0], res.degeneracy, 0.001)

We then pass the first (and largest) network in the system to the `draw_network` function.

This function makes sure that all molecules are "whole", i.e. their bonds are not split over periodic boundaries, and then moves all molecules between periodic images to minimise the distance between their centers of mass (this is done using a minimum spanning tree).
The end result is the most compact representation.

Finally the network is drawn over the molecules as red lines between the centers of electronically coupled molecules.

In [ ]:
v = kgp.visualise.draw_network(nets[0], show_molecules=True)
v.add_unitcell()
v

## Visualising a particular dimer

We might also be interested in visualising a particular dimer pair, perhaps because of their strong coupling.

In [ ]:
print("The largest coupling is: {:.3f} eV".format(res.H_frag[0].max()))

Here we determine the position in the coupling matrix of the largest coupling value:

In [ ]:
pos = np.argmax(res.H_frag[0])

x, y = pos // res.H_frag.shape[1], pos % res.H_frag.shape[2]

In [ ]:
res.H_frag[0, x, y]

We can then draw the fragments by passing these fragments (indexed by the variables `x` and `y`) to the `visualise.draw_fragments` functions.

In [ ]:
v = kgp.visualise.draw_fragments(u.atoms.fragments[x], u.atoms.fragments[y])
v.add_unitcell()
v